<a href="https://colab.research.google.com/github/harshalkondke/Hand-written-no-rec/blob/master/HandWrittenNoREC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# will download the database of images from MNIST website
import gzip
import os
import numpy as np


def load_dataset():
    def download(filename, source='http://yann.lecun.com/exdb/mnist/'):
        print ("Downloading", filename)
        import urllib.request
        urllib.request.urlretrieve(source+filename,filename)
        # will store the downloaded file in local disk

    def load_mnist_images(filename):
        if not os.path.exists(filename):
            download(filename)
        # check if file is available if not will download it
        
        with gzip.open(filename, 'rb') as f:
            # open the zp file of images

            data = np.frombuffer(f.read(), np.uint8, offset=16)

            data = data.reshape(-1, 1, 28, 28)
            # the first dimension is the number of images, by making this -1
            # second dimension is number of channels here it is 1 bcs its monocom image
            # the third and fourth dimension are the size if the image 28x28
            return data/np.float32(256)

    def load_mnist_labels(filename):
        if not os.path.exists(filename):
            download(filename)

        with gzip.open(filename, 'rb') as f:
            data = np.frombuffer(f.read(), np.uint8, offset=8)

        return data

    X_train = load_mnist_images('train-images-idx3-ubyte.gz')
    y_train = load_mnist_labels('train-labels-idx1-ubyte.gz')
    X_test = load_mnist_images('t10k-images-idx3-ubyte.gz')
    y_test = load_mnist_labels('t10k-labels-idx1-ubyte.gz')

    return X_train, y_train, X_test, y_test

In [0]:
X_train, y_train, X_test, y_test = load_dataset()

In [0]:
import matplotlib.pyplot as plt
plt.show(plt.imshow(X_train[2][0]))

if lasagne is not present in your colab, exicute this line of code

In [0]:
!pip install git+https://github.com/Lasagne/Lasagne
!pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git --user

In [0]:
import lasagne
import theano
import theano.tensor as T

def build_NN(input_var=None):
  # creating neural network with 2 hidden layers of 800 nodes each
  l_in = lasagne.layers.InputLayer(shape=(None, 1, 28, 28), input_var=input_var)
  l_in_drop = lasagne.layers.DropoutLayer(l_in,p=0.2)
  
  l_hid1 = lasagne.layers.DenseLayer(l_in_drop, num_units=800, nonlinearity=lasagne.nonlinearities.rectify, W=lasagne.init.GlorotUniform())
  l_hid1_drop = lasagne.layers.DropoutLayer(l_hid1, p=0.5)
  
  l_hid2 = lasagne.layers.DenseLayer(l_hid1_drop, num_units=800, nonlinearity=lasagne.nonlinearities.rectify, W=lasagne.init.GlorotUniform())
  l_hid2_drop = lasagne.layers.DropoutLayer(l_hid2, p=0.5)
  
  l_out = lasagne.layers.DenseLayer(l_hid2_drop, num_units=10, nonlinearity=lasagne.nonlinearities.softmax)
  
  return l_out 
  

In [0]:
input_var = T.tensor4('inputs') # 4d empty array
target_var = T.ivector('targets') # 2d linear array

network = build_NN(input_var)

prediction = lasagne.layers.get_output(network)
loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)

loss = loss.mean()

params = lasagne.layers.get_all_params(network, trainable=True)
updates = lasagne.updates.nesterov_momentum(loss, params, learning_rate=0.01, momentum = 0.9)

train_fn = theano.function([input_var, target_var], loss, updates=updates)


In [0]:
num_training_steps = 100

for step in range(num_training_steps):
  train_err = train_fn(X_train, y_train)
  print("Current step is "+str(step))

Current step is 0
Current step is 1
Current step is 2
Current step is 3
Current step is 4
Current step is 5
Current step is 6
Current step is 7
Current step is 8
Current step is 9
Current step is 10
Current step is 11
Current step is 12
Current step is 13
Current step is 14
Current step is 15


In [0]:
test_prediction = lasagne.layers.get_output(network)
val_fn = theano.function([input_var], test_prediction)

val_fn([X_test[2]])

check if the pridicted label is same as its original 

In [0]:
y_test[2]

In [0]:
test_prediction = lasagne.layers.get_output(network, deterministic=True)
test_acc = T.mean(T.eq(T.argmax(test_prediction, axis=1), target_var), dtype=theano.config.floatX)

acc_fn = theano.function([input_var, target_var], test_acc)

acc_fn(X_test, y_test)